In [52]:
import csv
import os
from datetime import datetime

def append_unique_to_csv(names,current_date, filename):
    # Check if the CSV file exists
    file_exists = os.path.isfile(filename)

    # Get the current date as a string (e.g., '2023-10-21')
    
    # Step 1: Add names to the CSV file
    existing_names = set()

    if file_exists:
        with open(filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                existing_names.add(row['names'])

    new_names = [name for name in names if name not in existing_names]

    if new_names:
        with open(filename, 'a', newline='') as csvfile:
            fieldnames = ['names']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            if not file_exists:
                writer.writeheader()

            for name in new_names:
                writer.writerow({'names': name})

    # Step 2: Add the current date as a new column header in ascending order
    existing_dates = set()
    fieldnames = []

    if file_exists:
        with open(filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            fieldnames = reader.fieldnames
            for fieldname in fieldnames:
                if fieldname != 'names':
                    existing_dates.add(fieldname)

    existing_dates = sorted(existing_dates)  # Sort the existing dates

    if current_date not in existing_dates:
        existing_dates.append(current_date)  # Add the current date
        existing_dates = sorted(existing_dates)  # Sort the dates again

        fieldnames = ['names'] + existing_dates

        with open(filename, 'r', newline='') as csvfile:
            # Read the existing data and then append the new header
            data = list(csv.DictReader(csvfile))

        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write the new header
            writer.writeheader()

            # Write the existing data back to the file
            writer.writerows(data)

def mark_attendance(names, date, filename):
    # Check if the CSV file exists
    file_exists = os.path.isfile(filename)

    # Step 1: Create a set of existing names and dates
    existing_names = set()
    existing_dates = set()

    if file_exists:
        with open(filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                existing_names.add(row['names'])
                for fieldname in reader.fieldnames:
                    if fieldname != 'names':
                        existing_dates.add(fieldname)

    # Step 2: Add the specified date if it doesn't exist
    if date not in existing_dates:
        existing_dates.add(date)

        # Sort the existing dates
        existing_dates = sorted(existing_dates)

        # Update the CSV file with the new header
        with open(filename, 'r', newline='') as csvfile:
            data = list(csv.DictReader(csvfile))

        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['names'] + existing_dates
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            # Write the existing data back to the file
            writer.writerows(data)

    # Step 3: Mark attendance for each name under the specified date
    with open(filename, 'r', newline='') as csvfile:
        data = list(csv.DictReader(csvfile))

    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['names'] + sorted(existing_dates)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        for name in names:
            if name in existing_names:
                row = next(row for row in data if row['names'] == name)
                row[date] = '1'
            else:
                # If the name is not in the CSV, create a new row with the name
                row = {'names': name}
                for d in existing_dates:
                    row[d] = '0'  # Initialize other dates with '0'
                row[date] = '1'  # Mark attendance for the specified date
                data.append(row)

        writer.writeheader()
        writer.writerows(data)

# Example usage:
current_date = '2023-10-25'#datetime.now().strftime('%Y-%m-%d')
name_list = ['face1','face2','face3','face4','face5','face6','face7','face8','face9','face10']
append_unique_to_csv(name_list,current_date, 'names.csv')

In [59]:
current_date = '2023-10-25'#datetime.now().strftime('%Y-%m-%d')
name_list = ['face1','face2','face3','face9','face10']
mark_attendance(name_list, current_date, 'names.csv')